In [1]:
%cd ../../..


/home/nazif/thesis/mirscribe


In [ ]:
import pandas as pd
from scripts.utils import *
from scripts.ensembl import *

hg38 = import_pyensembl(38)
hg19 = import_pyensembl(37)

In [ ]:
def read_gff_file(file_path):
    # Read the gff file into a DataFrame
    df = pd.read_csv(file_path, sep="\t", header=None)

    # Create a rename dictionary
    rename_dict = {
        0: 'chr',
        1: 'drop',
        2: 'biotype',
        3: 'start',
        4: 'end',
        5: 'drop2',
        6: 'strand',
        7: 'drop3',
        8: 'other_data'
    }

    # Rename the columns based on the rename dictionary
    df = df.rename(columns=rename_dict)

    df[['id', 'alias', 'name', 'derives_from']] = (df["other_data"].str.split(';', expand=True)
                                                   .apply(lambda x:x.str.split('=', expand=True)[1]))

    df['chr'] = df['chr'].str.replace('chr', '')

    df.drop(columns=["drop", "drop2", "drop3", "other_data"], inplace=True)

    return df


In [ ]:
raw = read_gff_file("scripts/convert_coordinates_pipeline/hsa.gff3")
converted_mirnas = read_gff_file("scripts/convert_coordinates_pipeline/hsa_mirnas_grch37.gff3")
non_converted_mirnas = read_gff_file("scripts/convert_coordinates_pipeline/hsa_mirnas_grch37.gff3.T99YAPR8.unmap")
mirbase = pd.read_csv(MIRNA_CSV)

# dropping non-mature miRNA entries
converted_mirnas = converted_mirnas.query("biotype == 'miRNA'")
non_converted_mirnas = non_converted_mirnas.query("biotype == 'miRNA'")

# dropping columns that are not needed in final file
converted_mirnas = converted_mirnas[["alias", "name", "chr", "start", "end", "strand"]]

# adding MIMAT0027619 to converted mirnas because its position is in the same in both grch37 and grch38
mirna = non_converted_mirnas.loc[1]
converted_mirnas.loc[len(converted_mirnas)] = mirna

# renaming
rename_dict = {
    "alias": "mirna_accession",
    "name": "mirna_name"
}
converted_mirnas = converted_mirnas.rename(columns=rename_dict)

# saving to csv
converted_mirnas.to_csv("data/mirna_coordinates/grch37_coordinates.csv", index=False)